In [1]:
import findspark
findspark.init()

In [2]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
data = spark.read.csv("Absenteeism_at_work.csv", inferSchema=True, header = True, sep=",")

In [5]:
data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Reason for absence: integer (nullable = true)
 |-- Month of absence: integer (nullable = true)
 |-- Day of the week: integer (nullable = true)
 |-- Seasons: integer (nullable = true)
 |-- Transportation expense: integer (nullable = true)
 |-- Distance from Residence to Work: integer (nullable = true)
 |-- Service time: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Work load Average/day : double (nullable = true)
 |-- Hit target: integer (nullable = true)
 |-- Disciplinary_failure: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- Son: integer (nullable = true)
 |-- Social drinker: integer (nullable = true)
 |-- Social smoker: integer (nullable = true)
 |-- Pet: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Body mass index: integer (nullable = true)
 |-- Absenteeism_time_in_hours: integer (nullable = true)
 |-- MOA: integer (nullable = t

In [4]:
data = data.withColumn("MOA", data["Month of absence"]). \
            withColumn("label", data['Height']). \
            withColumn("ROA", data["Reason for absence"]). \
            withColumn("distance", data["Distance from Residence to Work"]). \
            withColumn("BMI", data["Body mass index"])

In [7]:
from pyspark.ml.feature import VectorAssembler
assem = VectorAssembler(inputCols=["label", "MOA"], outputCol='features')

In [8]:
data = assem.transform(data)

In [6]:
from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel")

In [9]:
from pyspark.ml.feature import VectorIndexer
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4)

In [10]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3],seed=1)

In [29]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [30]:
from pyspark.ml import Pipeline
# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, nb])#, labelConverter])

In [31]:
# train the model
model = pipeline.fit(trainingData)

In [32]:
# select example rows to display.
predictions = model.transform(testData)

In [33]:
# Select example rows to display.
predictions.select("prediction", "indexedlabel", "features").show()

+----------+------------+------------+
|prediction|indexedlabel|    features|
+----------+------------+------------+
|       0.0|         1.0| [172.0,1.0]|
|       0.0|         1.0| [172.0,3.0]|
|       1.0|         1.0| [172.0,8.0]|
|       1.0|         1.0|[172.0,12.0]|
|       1.0|         1.0|[172.0,12.0]|
|       0.0|        12.0| [163.0,6.0]|
|       1.0|        12.0| [163.0,8.0]|
|       0.0|         0.0| [170.0,2.0]|
|       0.0|         0.0| [170.0,6.0]|
|       0.0|         0.0| [170.0,7.0]|
|       1.0|         0.0| [170.0,8.0]|
|       1.0|         0.0| [170.0,9.0]|
|       0.0|         0.0| [170.0,2.0]|
|       0.0|         0.0| [170.0,2.0]|
|       1.0|         0.0| [170.0,9.0]|
|       0.0|         0.0| [170.0,5.0]|
|       0.0|         0.0| [170.0,2.0]|
|       0.0|         0.0| [170.0,7.0]|
|       0.0|         0.0| [170.0,1.0]|
|       0.0|         0.0| [170.0,2.0]|
+----------+------------+------------+
only showing top 20 rows



In [34]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel",\
                                              predictionCol="prediction",\
                                              metricName="accuracy")

In [21]:
y_true = data.select("BMI").rdd.flatMap(lambda x: x).collect()
y_pred = data.select("ROA").rdd.flatMap(lambda x: x).collect()

In [35]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

nbModel = model.stages[2]

In [36]:
print(nbModel)  # summary only
print("Naive Bayes - Test Accuracy = %g" % (accuracy))
print("Naive Bayes - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Naive Bayes Model is :\n" + str(confusionmatrix))

print("The precision score for Naive Bayes Model is: " + str(precision))

print("The recall score for Naive Bayes Model is: " + str(recall))

NaiveBayes_6743726eb968
Naive Bayes - Test Accuracy = 0.272321
Naive Bayes - Test Error = 0.727679
The Confusion Matrix for Naive Bayes Model is :
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]]
The precision score for Naive Bayes Model is: 0.02972972972972973
The recall score for Naive Bayes Model is: 0.02972972972972973
